In [1]:
#Importamos las librerias
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import time
from numpy import linalg as LA
import random
from time import time
%matplotlib qt

In [2]:
#Definimos la funcion a minimizar
def f(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [3]:
#Definir funcion para calcular la matriz del gradiente
def gradient(x,delta):
    grad=np.zeros(2)
    grad[0]=(f([x[0]+delta,x[1]])- f([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(f([x[0],x[1]+delta])- f([x[0],x[1]-delta]))/(2*delta)
    return grad

In [4]:
#Definir funcion para calcular la matriz Hessiana
def Hessian(x,delta):
    H=np.zeros([2,2])
    H[0,0]= (f([x[0]+delta,x[1]]) - 2*f([x[0],x[1]]) +  f([x[0]-delta,x[1]]))/(delta**2)
    H[1,1]= (f([x[0],x[1]+delta]) - 2*f([x[0],x[1]]) +  f([x[0],x[1]-delta]))/(delta**2)
    H[0,1]= (f([x[0]+delta,x[1]+delta]) - f([x[0]+delta,x[1]-delta]) - f([x[0]-delta,x[1]+delta]) +  f([x[0]-delta,x[1]-delta]))/(4*delta**2)
    H[1,0]= H[0,1]
    return H

In [5]:
#Utilizamos el algoritmo de golden section para encontrar alpha optimo
def golden(x,search,xi,eps):
    a = xi[0];
    b = xi[1];
    tau = 0.381967;
    alpha1 = a*(1-tau) + b*tau;
    alpha2 = a*tau + b*(1-tau);
    falpha1 = f(x+alpha1*search);
    falpha2 = f(x+alpha2*search);
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1;
            alpha1 = alpha2;
            falpha1 = falpha2;
            alpha2 = tau*a + (1-tau)*b;
            falpha2 = f(x+alpha2*search);
        else:
            b = alpha2;
            alpha2 = alpha1;
            falpha2 = falpha1;
            alpha1 = tau*b + (1-tau)*a;
            falpha1 = f(x+alpha1*search);
        if np.abs(f(x+alpha1*search)- f(x+alpha2*search)) < eps :
            break;
    return alpha1,falpha1

In [6]:
#Graficamos la funcion de prueba
X1 = np.arange(-5, 5, 0.01)
X2 = np.arange(-5, 5, 0.01)
x1, x2 = np.meshgrid(X1, X2)
z = f([x1,x2])
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
surf = ax.plot_surface(x1, x2, z, cmap=cm.autumn, linewidth=0, antialiased = False, alpha=0.2)
plt.show()

In [7]:
#Variable en comun de los métodos
delta = 1e-3 
ep1 = 1e-3
puntosGC = []

In [8]:
def graficaUnaaUna(puntos, nombreMetodo):
    #Generamos valores ramdom para generar un color diferente de grafica
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b)
    aux = 0
    x = np.linspace(-1, 1, 100)
    y = np.linspace(-1, 1, 100)
    xx, yy = np.meshgrid(x, y)
    z = f([xx, yy])
    plt.contour(x,y,z,20)
    plt.plot(-1,-1, 'ro--',  markersize=6)
    plt.plot(0.5,0, 'go--', markersize=6)
    #Graficamos los trazos (direcciones) que va generando el algoritmo
    for pts in puntos:
        if(aux==len(puntos)-1):
            plt.plot(pts[0],pts[1], c=color, linewidth=2, label=str(nombreMetodo))
        else:
            plt.plot(pts[0],pts[1], c=color, linewidth=2)
        aux = aux + 1
    plt.legend()    
    plt.show

In [9]:
start_time = time() #Conteo para tiempo
iteraciones = 0 #Conteo de numero de iteraciones
x = [-1,1] #Punto de partida
x_prev1 = x #Definimos una variable de punto anterior
x_prev = x #Definimos una variable de punto anterior
f_prev = f(x) #Se evalua la funcion con el punto inicial
print('Valor de la función inicial = %f ' % f_prev)
dire_prev=gradient(x,delta)
si_prev = -dire_prev
#x,search,xi,eps
alpha, fx_prev = golden(x,si_prev,x_prev,ep1);
x = x +  alpha*si_prev 
for i in range(1000): #Proceso iterativo del algoritmo
    dire=gradient(x,delta)
    si = - dire +((LA.norm(dire)**2)/(LA.norm(dire_prev)**2))*si_prev;
    alpha, fx_curr = golden(x,si,x_prev,ep1);
    if abs(fx_curr-fx_prev)<ep1 or LA.norm(dire)<ep1:
        print('{0}\t[{1:.3f},{2:.3f}]\t{3:.3f}\t{4:.3f}'.format(i, x[0], x[1],fx_curr,LA.norm(dire)))
        break;
    dire_prev=dire
    si_prev=si
    x = x +  (alpha*si).transpose() 
    fx_prev=f(x)
    print('{0}\t[{1:.3f},{2:.3f}]\t{3:.3f}\t{4:.3f}'.format(i, x[0], x[1],fx_curr,LA.norm(dire)))
    puntosGC.append([[x_prev1[0],x[0]],[x_prev1[1],x[1]]])
    x_prev1 = x
    iteraciones+=1
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosGC, 'Conjugate gradient method')  

Valor de la función inicial = 270.294169 
0	[0.030,0.199]	-1.026	33.677
1	[0.438,0.101]	-6.301	39.919
2	[0.502,-0.001]	-7.353	26.034
3	[0.502,-0.001]	-7.353	0.224
Tiempo transcurrido: 2.989531
Iteraciones: 3


NameError: name 'times' is not defined